<a href="https://colab.research.google.com/github/duyvm/leetcode-problems/blob/main/%5BHAR%5D2999_Count_the_Number_of_Powerful_Integers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2999. Count the Number of Powerful Integers

https://leetcode.com/problems/count-the-number-of-powerful-integers/description/

You are given three integers `start`, `finish`, and `limit`. You are also given a **0-indexed** string `s` representing a positive integer.

A positive integer `x` is called powerful if it ends with `s` (in other words, `s` is a suffix of `x`) and each digit in `x` is at **most limit**.

Return the total number of powerful integers in the range `[start..finish]`.

A string `x` is a suffix of a string `y` if and only if `x` is a substring of `y` that starts from some index (including `0`) in `y` and extends to the index `y.length - 1`. For example, `25` is a suffix of `5125` whereas `512` is not.

**Constraints:**
- `1 <= start <= finish <= 10`<sup>`15`</sup>
- `1 <= limit <= 9`
- `1 <= s.length <= floor(log`<sub>`10`</sub>`(finish)) + 1`
- `s` only consists of numeric digits which are at most `limit`.
- `s` does not have leading zeros.

**Follow up:**

**Example 1:**

- Input: `"start": 1, "finish": 6000, "limit": 4, "s": "124"`

- Output: `5`

- Explanation:

        The powerful integers in the range [1..6000] are 124, 1124, 2124, 3124, and, 4124. All these integers have each digit <= 4, and "124" as a suffix. Note that 5124 is not a powerful integer because the first digit is 5 which is greater than 4.
        It can be shown that there are only 5 powerful integers in this range.

**Example 2:**

- Input: `"start": 15, "finish": 215, "limit": 6, "s": "10"`

- Output: `2`

- Explanation:

        The powerful integers in the range [15..215] are 110 and 210. All these integers have each digit <= 6, and "10" as a suffix.
        It can be shown that there are only 2 powerful integers in this range.

**Example 3:**

- Input: `"start": 1000, "finish": 2000, "limit": 4, "s": "3000"`

- Output: `5`

- Explanation: 0

        All integers in the range [1000..2000] are smaller than 3000, hence "3000" cannot be a suffix of any integer in this range.

In [13]:
test_cases = [
    {
        "input": {
            "start": 1, "finish": 6000, "limit": 4, "s": "124"
        },
        "output": 5
    },
    {
        "input": {
            "start": 15, "finish": 215, "limit": 6, "s": "10"
        },
        "output": 2
    },
    {
        "input": {
            "start": 1000, "finish": 2000, "limit": 4, "s": "3000"
        },
        "output": 0
    },
    {
        "input": {
            "start": 3, "finish": 1429, "limit": 5, "s": "11"
        },
        "output": 11
    },
    {
        "input": {
            "start": 1000000, "finish": 123000000000000, "limit": 6, "s": "41"
        },
        "output": 18643364033
    },
    {
        "input": {
            "start": 1, "finish": 971, "limit": 9, "s": "72"
        },
        "output": 9
    },
    {
        "input": {
            "start": 10, "finish": 1844, "limit": 5, "s": "12"
        },
        "output": 12
    },
]

In [3]:
def run_test_cases(solution, function_name):
    for i in range(len(test_cases)):
        run_test_case(solution, function_name, i)

def run_test_case(solution, function_name, i):
    test_case = test_cases[i]
    result = getattr(solution, function_name)(**test_case["input"])
    if result != test_case["output"]:
        print(f'Failed test case {i} with input: {test_case["input"]} and expected result: {test_case["output"]} vs actual result: {result}')

# Approach 1 - Digit Dynamic Programing (Hint from Leetcode) - Time Limit Exceeded

### Observations

- Reference for digit DP: https://codeforces.com/blog/entry/53960

- Find the number of powerful positive in range `[1:start-1]`. Let it `start_count`

- Find the number of powerful positive in range `[1:finish]`. Let it `finish_count`

- The number of powerfull positive in range `[start:finish]` is `finish_count - start_count`

- Using digit DP to find number of powerful pisitive in range `[1:x]`:

  - Start from digit left to digit right of a number with length equal `x.length`, including leading zeros.

  - Let `sq` is the number that we are considering. `sq.length == x.length`

  - The left most digit's position is `1`

  - The right most digit's position is `x.length`

  - For current `pos`, we add new digit to the `sq[pos]`
  
    - We must consider to check if adding digit `i` to current `pos` make the number larger than `x`. We check that for `1 <= t < pos`, if there exists any `sq[t] < x[t]`, then we can saftely say that adding any digit `i` will not make `sq > x`. We count the number of possible `i` and proceed to next recursion round `pos + 1`

    - We do not actually need to checking all digit `i` for current `pos`, we use a variable `isSmallerThanX` to track if current `sq` smaller than `x` or not. Then we pass it to the next recursion round along with `pos+1`.

    - Because we start from the left most digit, which is the most significant digit of `sq`, from the `pos = 1` we already know if current `sq` is larger than `x` or not and pass the information to the next recursion round.

    - Starting the recursion round, if `isSmallerThanX` is `false` then we can only add digit `i <= limit`. If `x[pos] < limit`, then if we add `i < nums[pos]`, the current number is smaller than `x`. We still need to consider case `i = x[pos]` while current number is equal to `x`

- We need to consider range`[max(s, start):finish]`

### Analysis

- Time complexity:


### Implementation

In [11]:
from re import L
from typing import List, Optional
from collections import defaultdict
from collections import deque
from heapq import heapify, heappush, heappop
from queue import PriorityQueue
from collections import Counter
import math

class Solution:
    def numberOfPowerfulInt(self, start: int, finish: int, limit: int, s: str) -> int:
        if int(s) > finish:
            return 0
        if int(s) == finish:
            return 1

        # upper bound
        numberOfDigitNeedToBeConsider = len(str(finish)) - len(s)
        # print(f"upper bound: {str(finish), 0, False, limit, numberOfDigitNeedToBeConsider}")
        finish_count = self.digitDP(str(finish), 0, False, limit, numberOfDigitNeedToBeConsider, "", s)

        # lower bound
        start = start if start >= int(s) else int(s)
        numberOfDigitNeedToBeConsider = len(str(start)) - len(s)
        # print(f"lower bound: {str(start-1), 0, False, limit, numberOfDigitNeedToBeConsider}")
        start_count = self.digitDP(str(start-1), 0, False, limit, numberOfDigitNeedToBeConsider, "", s)

        return finish_count - start_count


    def digitDP(self, x, pos, isSmallerThanX, limit, numberOfDigitNeedToBeConsider, sq, s):
        # end of sq
        if pos == numberOfDigitNeedToBeConsider:
            if isSmallerThanX or int(sq+s) <= int(x):
                # print(f"valid sq: {sq+s}")
                return 1
            else:
                # print(f"invalid sq: {sq+s}")
                return 0

        count = 0


        if isSmallerThanX or limit < int(x[pos]):
            # any digit is possible
            for i in range(0, limit+1):
                count += self.digitDP(x, pos + 1, True, limit, numberOfDigitNeedToBeConsider, sq+str(i), s)
        else:
            for i in range(0, int(x[pos])):
                count += self.digitDP(x, pos + 1, True, limit, numberOfDigitNeedToBeConsider, sq+str(i), s)

            count +=  self.digitDP(x, pos + 1, False, limit, numberOfDigitNeedToBeConsider, sq+x[pos], s)

        return count

In [12]:
run_test_cases(Solution(), "numberOfPowerfulInt")

# Approach 2 - Combinatorial mathematics (Beat 95%)

### Observations

- Same with approach 1. To find the number of powerful integers in range `[start:finish]`, we need to find the number of powerful integers in range `[1:max(start, s)-1]` and `[1:finsh]`. The subtract the former from the later.

- To calculate number of powerful integers in range `[1:x]`, instead of using digit DB, we can directly calculate it by consider the number of digits we need to fill in and and possible digits that are possible in the position.

- Let `pre_len = len(x) - len(s)` is the number of digits that we need to fill in. If `pre_len < 0`, it means that `x < s` and there are not powerful integers in range `[1:x]`. We return `0`

- Now, we consider the each digit in range of `x[:pre_len]:`

  - Let `sequence` is the number that we are considering. `sequence.length = x.length`. Leading zeroes is possible. The suffix `sequence[pre_len:]` is fixed, which is `s`.

  - For current index `0 <= i < pre_len`
  
  - If `x[i] > limit`, it means we can fill in `sequence[i]` any digits that satify limit. And from index `i+1` to `pre_len-1`, we do not need to consider each digit `x[i+1:pre_len]` anymore, because at `i`, we already know that `sequence[i:] < x[i:]`.

    - We directly calculate all possible powerful integers by formula:
    
          (limit+1)^(pre_len-1)

    - For exmaple: let `x = 178999`, `s = 9` and considering `sequence = 1____9`, `limit = 5`. At index `i=1`, `x[1] = 7 > limit = 5`, so we can only fill in `sequence[1]` with value from `0` to `5` and we know that from that index onward, `sequence` already smaller than `x`. We do not need to consider all the index in range `2` to `4`. NUmber of possible choices for digit in index `2` to `4` is:
    
          (limit + 1) * (limit + 1) * (limit + 1) = (limit + 1) ^ 3

  - If `x[i] <= limit`: we need to consider digit from `0` up to `x[i]`

    - With digit from `0` to `x[i] - 1`, same as `x[i] > limit` case, `sequence[i:]` alread smaller than `x[i:]`, we directly calculate all possible powerful integers by formula:

          x[i] * (limit+1) ** (pre_len-i-1)
          
          we mutiply with x[i] because we consider from 0 to x[i]

    - We need to consider case `sequence[i] = x[i]`: if we fill `i` with digit `x[i]`, then up to index `i`, we filled sequence with with value same as `x`. We move to next index `i+1` to continue to check again.
      
      - For example `x = 123456` and `sequence = `12__34`, `limit = 8` and we are at index `i=1`. We move to `i=2` and checking again with the same logic
    
      - At the end of considering index `x[pre_len-1]`, if `sequence[pre_len-1] = x[pre_len-1]` then if suffix of `x[pre_len:] > s`, it contributes `1` to the final count. With the above example, suffix of `x = 56` is larger than `s = 34`. The whole number at final is `x=123456` and `sequence = 123434`, then `sequence` is a powerful integer.

### Analysis

- Time complexity:

  - One pass through all digits in `finish`: `O(log`<sub>`10`</sub>`(finish))`

  - One pass through `start` or `s`. Since `finish > start/s`, time complexity will be smaller than `O(log`<sub>`10`</sub>`(finish))`

  - Total time complexity: `O(log`<sub>`10`</sub>`(finish))`

In [22]:
from re import L
from typing import List, Optional
from collections import defaultdict
from collections import deque
from heapq import heapify, heappush, heappop
from queue import PriorityQueue
from collections import Counter
import math

class Solution:
    def numberOfPowerfulInt(self, start: int, finish: int, limit: int, s: str) -> int:
        if int(s) > finish:
            return 0
        if int(s) == finish:
            return 1

        # upper bound
        finish_count = self.calculate(str(finish), limit, s)

        # lower bound
        start = max(start, int(s))
        start_count = self.calculate(str(start-1), limit, s)

        return finish_count - start_count


    def calculate(self, x, limit, s):
        pre_len = len(x) - len(s)
        count = 0

        sq = ""

        for i in range(pre_len):
            if limit < int(x[i]):
                return count + (limit+1) ** (pre_len-i)
            else:
                count += int(x[i]) * (limit+1) ** (pre_len-i-1)

                sq += x[i]

        if int(s) <= int(x[pre_len:]):
            count += 1
        return count

In [23]:
run_test_cases(Solution(), "numberOfPowerfulInt")